In [ ]:
# Install  Packets

import sys
!{sys.executable} -m pip install lightfm
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install surprise
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install scikit-surprise 
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install google.colab
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install fastparquet
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install pylance
!{sys.executable} -m pip install python-Levenshtein


In [ ]:
# To store the data
import pandas as pd

# To do linear algebra
import numpy as np

# To create plots
import matplotlib.pyplot as plt
import seaborn as sns

# To create interactive plots
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# To shift lists
from collections import deque

# To compute similarities between vectors
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# To use recommender systems
import surprise as sp
from surprise.model_selection import cross_validate

# To create deep learning models
from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

# To create sparse matrices
from scipy.sparse import coo_matrix

# To light fm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

# To stack sparse matrices
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from fuzzywuzzy import fuzz

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [ ]:
#Als Nächstes sollen alle Pfade mit variablen belegt werden, dass macht das austauschen einfacher.

movie_tile_File = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/movie_titles.csv'
movie_tile_File_new = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Titles_clear.gzip'
combined_data_1 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_1.txt'
combined_data_2 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_2.txt'
combined_data_3 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_3.txt'
combined_data_4 = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/combined_data_4.txt'
new_Combined = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Combined_Data_All.gzip'
netflix_rating_Combined = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/netflix_data.csv'
parquet_combined_data = 'C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/data_Comb.zip'

In [ ]:
data = pd.read_parquet(new_Combined)
Convert_dic_data= {'Cust_Id': 'int64', 'Rating': 'float32', 'Movie_Id': 'Int64'}
data= data.astype(Convert_dic_data)
del new_Combined

In [39]:
movie_titles = pd.read_csv(movie_tile_File,
                           encoding = "ISO-8859-1",
                           delimiter= '\t',
                           header = None,
                           names = ['Target'])
                           # Speicher alle Daten erst in eine Reihe, danach trennt er diese
movie_titles[['Movie_Id', 'Year', 'Name']] = movie_titles['Target'].str.split(pat=",",n=2, expand=True)   
movie_titles= movie_titles.drop(['Target', 'Year'], axis= 1)  
Convert_dic= {'Movie_Id': 'int64', 'Name': 'str'}
movie_titles = movie_titles.astype(Convert_dic)
#movie_titles.astype({'Movie_Id': 'int64', 'Name': 'str'}).dtypes
#movie_titles.rename(columns={'Id':'Movie_Id'}, inplace=True)
#movie_titles = movie_titles.reset_index(drop=True)
display(movie_titles)

del Convert_dic

,Movie_Id,Name
0,1,Dinosaur Planet
1,2,Isle of Man TT 2004 Review
2,3,Character
3,4,Paula Abdul's Get Up & Dance
4,5,The Rise and Fall of ECW
...,...,...
17765,17766,Where the Wild Things Are and Other Maurice Se...
17766,17767,Fidel Castro: American Experience
17767,17768,Epoch
17768,17769,The Company


In [ ]:
num_users = len(data.Cust_Id.unique())
num_items = len(data.Movie_Id.unique())
print('There are {} unique users and {} unique movies in this data set'.format(num_users, num_items))

In [ ]:
df_ratings_cnt_tmp = pd.DataFrame(data.groupby('Rating').size(), columns=['count'])
df_ratings_cnt_tmp

In [ ]:
data = data.reset_index(drop=True)
print(data.head(3))

In [ ]:
# get rating frequency
df_movies_cnt = pd.DataFrame(data.groupby('Movie_Id').size(), columns=['count'])
df_movies_cnt.head()
df_movies_cnt['count'].quantile(np.arange(1, 0.6, -0.05))

In [ ]:
# filter data
popularity_thres = 17700
popular_movies = list(set(df_movies_cnt.query('count >= @popularity_thres').index))
df_ratings_drop_movies = data[data.Movie_Id.isin(popular_movies)]
print('shape of original ratings data: ', data.shape)
print('shape of ratings data after dropping unpopular movies: ', df_ratings_drop_movies.shape)

In [ ]:
# get number of ratings given by every user
df_users_cnt = pd.DataFrame(df_ratings_drop_movies.groupby('Cust_Id').size(), columns=['count'])
df_users_cnt.head()

In [ ]:
df_users_cnt['count'].quantile(np.arange(1, 0.5, -0.05))

In [ ]:
# filter data
ratings_thres = 500
active_users = list(set(df_users_cnt.query('count >= @ratings_thres').index))
df_ratings_drop_users = df_ratings_drop_movies[df_ratings_drop_movies.Cust_Id.isin(active_users)]
print('shape of original ratings data: ', data.shape)
print('shape of ratings data after dropping both unpopular movies and inactive users: ', df_ratings_drop_users.shape)

In [ ]:
df = data
# Filter sparse movies
min_movie_ratings = 1000
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]

print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))
del filter_movies, filter_users, min_movie_ratings, min_user_ratings, data, df

In [ ]:
df_movie_id_list = df_filterd.Movie_Id.unique().tolist()
df_movie_id_list = pd.DataFrame(df_movie_id_list)
print(df_movie_id_list.head(4))
# Alle movie_Ids die überprüft werden müssen. Alle IDs, werden in ein DataFrame gespeichert

In [ ]:
# pivot and create movie-user matrix

movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)


In [ ]:
## 1.       Pivot Tabelle, wobei die Movie_IDs Index sind  ( list(movie_user_mat.index.values))
## 1.1.     Für jeden Film müssen die nächsten Nachbarn berechnet werden. 
#  1.2.     Die Iteration der Berechnung soll mit Hilfe des Index in der Tabelle movie_user_mat geschehen. 
#  2.       Für die nächsten nachbarn soll eine Liste erstellt werden. 
# 2.1.      Jede Berechnung bzw. die Ergebnisse sollen in eine neue Liste gepackt werden. Spricht alle Iterationen sollen in eine Liste zwischengespeichert werden und am Ende in eine CSV Exportiert werden.


#https://github.com/aniketng21/Movie-Recommendation-System-Using-KNN-Algorithm/blob/master/Movie_Recommendation_System.ipynb

In [ ]:
movie_index_list = list(movie_user_mat.index.values)
# Get 

In [ ]:
# We choose random movie.
#query_index = 8585
#query_index = (movie_user_mat.shape[0])
# Haiku Tunnel 6899 .95 Precision

query_index = np.random.choice(movie_user_mat.shape[0])



print("Choosen Movie is: ",movie_user_mat.index[query_index])

print(movie_user_mat.index[1])



In [ ]:
user_movie_table_matrix = csr_matrix(movie_user_mat.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'auto',n_jobs=-1)
model_knn.fit(user_movie_table_matrix)

In [ ]:

df = pd.DataFrame(columns=['Movie_Id', 'R_Movie_Id_1', 'R_Distance_1' , 'R_Movie_Id_2'  , 'R_Distance_2' , 'R_Movie_Id_3'  ,'R_Distance_3','R_Movie_Id_4' ,'R_Distance_4' ,'R_Movie_Id_5' ,'R_Distance_5'])
Convert_dic= {'Movie_Id': 'int32', 'R_Movie_Id_1': 'int32','R_Distance_1':'float','R_Movie_Id_2': 'int32','R_Distance_2':'float','R_Movie_Id_3': 'int32','R_Distance_3':'float','R_Movie_Id_4': 'int32','R_Distance_4':'float','R_Movie_Id_5': 'int32','R_Distance_5':'float' }
df = df.astype(Convert_dic)


In [ ]:
### Get Dataframe List

for movie_id in movie_index_list:
    #print(movie_id)
    movie_id_mat = movie_user_mat.loc[[movie_id]].values.reshape(1,-1)
    distances, indices = model_knn.kneighbors(movie_id_mat, n_neighbors = 120)
    movie = []
    distance = []
    

    for i in range(0, len(distances.flatten())):
        if i != 0:
            movie.append(movie_user_mat.index[indices.flatten()[i]])
            distance.append(distances.flatten()[i])    

    m=pd.Series(movie,name='Movie_Id')
    d=pd.Series(distance,name='distance')
    recommend = pd.concat([m,d], axis=1)
    recommend = recommend.sort_values('distance',ascending=False)


    df = df.append({'Movie_Id': movie_id, 'R_Movie_Id_1': recommend['Movie_Id'].iloc[0], 'R_Distance_1':recommend['distance'].iloc[0],
        'R_Movie_Id_2': recommend['Movie_Id'].iloc[1], 'R_Distance_2':recommend['distance'].iloc[1],
        'R_Movie_Id_3': recommend['Movie_Id'].iloc[2], 'R_Distance_3':recommend['distance'].iloc[2],
        'R_Movie_Id_4': recommend['Movie_Id'].iloc[3], 'R_Distance_4':recommend['distance'].iloc[3],
        'R_Movie_Id_5': recommend['Movie_Id'].iloc[4], 'R_Distance_5':recommend['distance'].iloc[4]
        },ignore_index=True)
    

In [ ]:
Convert_dic= {'Movie_Id': 'int32', 'R_Movie_Id_1': 'int32','R_Distance_1':'float','R_Movie_Id_2': 'int32','R_Distance_2':'float','R_Movie_Id_3': 'int32','R_Distance_3':'float','R_Movie_Id_4': 'int32','R_Distance_4':'float','R_Movie_Id_5': 'int32','R_Distance_5':'float' }
df = df.astype(Convert_dic)

In [44]:
df_rounded = df.round({'R_Distance_1': 4, 'R_Distance_2':4,'R_Distance_3': 4, 'R_Distance_4':4,'R_Distance_5':4 })

In [ ]:
df.to_parquet('C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Recommendations.zip',index= 'False', compression= 'gzip')
df.to_csv('C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Recommendations_header.csv',index= False,sep=';',header='True')
df_rounded.to_csv('C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Recommendations_header_rounded.csv',index= False, sep=';',header='True', decimal=',')
df.to_parquet('C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Recommendations_index.zip',index= 'True', compression= 'gzip')

In [46]:
print(df_rounded.head(2))

                   Movie_Id   R_Movie_Id_1  R_Distance_1       R_Movie_Id_2  \
0                 Character  Bad Education        0.8760  The Piano Teacher   
1  The Rise and Fall of ECW   American Pie        0.9313            Be Cool   

   R_Distance_2     R_Movie_Id_3  R_Distance_3       R_Movie_Id_4  \
0        0.8760  Kitchen Stories        0.8758  Felicia's Journey   
1        0.9313        They Live        0.9311          Rocky III   

   R_Distance_4                         R_Movie_Id_5  R_Distance_5  
0        0.8757                The Magdalene Sisters        0.8757  
1        0.9311  Dave Chappelle: Killin' Them Softly        0.9311  


In [47]:
df_rounded.to_csv('C:/Users/jsbreite\OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/Movie_Recommendations_header_rounded_name.csv',index= False, sep=';',header='True', decimal=',')

In [45]:
df_rounded['Movie_Id'] = df_rounded['Movie_Id'].map(movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_1'] = df_rounded['R_Movie_Id_1'].map(movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_2'] = df_rounded['R_Movie_Id_2'].map(movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_3'] = df_rounded['R_Movie_Id_3'].map(movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_4'] = df_rounded['R_Movie_Id_4'].map(movie_titles.set_index('Movie_Id')['Name'])
df_rounded['R_Movie_Id_5'] = df_rounded['R_Movie_Id_5'].map(movie_titles.set_index('Movie_Id')['Name'])
print(df_rounded)

                        Movie_Id  \
0                      Character   
1       The Rise and Fall of ECW   
2                           Sick   
3     What the #$*! Do We Know!?   
4                      Screamers   
...                          ...   
7120                     Gattaca   
7121                   Interiors   
7122         Shakespeare in Love   
7123                       Epoch   
7124                 The Company   

                                      R_Movie_Id_1  R_Distance_1  \
0                                    Bad Education        0.8760   
1                                     American Pie        0.9313   
2     The Cook, the Thief, His Wife, and Her Lover        0.9160   
3                                    The Godfather        0.8158   
4                                       Nightbreed        0.7984   
...                                            ...           ...   
7120                                 Almost Famous        0.6160   
7121                   

In [ ]:
## Not Good Code
#
#
movie_list  = {}
for movie_id in movie_index_list[:5]:
    #print(movie_id)
    movie_id_mat = movie_user_mat.loc[[movie_id]].values.reshape(1,-1)
    distances, indices = model_knn.kneighbors(movie_id_mat, n_neighbors = 60)
    movie_list[movie_id] = []
    movie = []
    distance = []
    

    for i in range(0, len(distances.flatten())):
        if i != 0:
            movie.append(movie_user_mat.index[indices.flatten()[i]])
            distance.append(distances.flatten()[i])    

    m=pd.Series(movie,name='Movie_Id')
    d=pd.Series(distance,name='distance')
    recommend = pd.concat([m,d], axis=1)
    recommend = recommend.sort_values('distance',ascending=False)

    print('Recommendations for {0}:\n'.format(movie_id))
    #test_list.append(movie_id, recommend['Movie_Id'].iloc[i], recommend['distance'].iloc[i],recommend['Movie_Id'].iloc[1+i], recommend['distance'].iloc[1+i])
   
    df = df.append({'Movie_Id': movie_id, 'R_Movie_Id_1': recommend['Movie_Id'].iloc[0], 'R_Distance_1':recommend['distance'].iloc[0],
        'R_Movie_Id_2': recommend['Movie_Id'].iloc[1], 'R_Distance_2':recommend['distance'].iloc[1],
        'R_Movie_Id_3': recommend['Movie_Id'].iloc[2], 'R_Distance_3':recommend['distance'].iloc[2],
        'R_Movie_Id_4': recommend['Movie_Id'].iloc[3], 'R_Distance_4':recommend['distance'].iloc[3],
        'R_Movie_Id_5': recommend['Movie_Id'].iloc[4], 'R_Distance_5':recommend['distance'].iloc[4]
        },ignore_index=True)
    


    for i in range(0,5):
        #df= df.append({'Movie_Id': movie_id})
        movie_list[movie_id].append({'Movie_Id': recommend['Movie_Id'].iloc[i], 'distance': recommend['distance'].iloc[i]})

        #df = pf.dataframe({'Movie_ID': [movie_id, recommend['Movie_Id'].iloc[i]]})
        
        print('{0}: {1}, with distance of {2}'.format(i, recommend["Movie_Id"].iloc[i], recommend["distance"].iloc[i]))


In [ ]:
print(df.dtypes)

In [ ]:
display(movie_list)
recommendation_list = pd.DataFrame(movie_list).transpose()
print(recommendation_list.head(3))

In [ ]:
print(movie_user_mat.iloc[query_index,:].values.reshape(1,-1))

print(movie_user_mat.index[[1406]])
print(movie_user_mat.iloc[[1406]].values.reshape(1,-1))
print(movie_user_mat.shape)
print(query_index)

#Heute aebnd verstehen


In [ ]:
#print(movie_user_mat.index[3,:].values.reshape(1,-1))
print("Choosen Movie is: ",movie_user_mat.index[query_index])
print(movie_user_mat.iloc[1406,:].values.reshape(1,-1))

print(movie_user_mat.iloc[query_index,:].values.reshape(1,-1))



In [ ]:
# Iteration aufbauen
distances, indices = model_knn.kneighbors(movie_user_mat.iloc[query_index,:].values.reshape(1,-1), n_neighbors = 60)

In [ ]:
distances, indices = model_knn.kneighbors(test_jannis, n_neighbors = 60)

In [ ]:
## Muss durch jeden Film ausgeführt werden Iteration ausgeführt werden.
distances, indices = model_knn.kneighbors(movie_user_mat, n_neighbors = 1)

In [ ]:
for a in Movie_index_list:
    print(a)

In [ ]:
movie_list = {}


for name_movie, values in movie_user_mat.iteritems():
    movie_list[name_movie] = []
    movie = []
    distance = []

    for i in range(0, len(distances.flatten())):
        if i != 0:
            movie.append(movie_user_mat.index[indices.flatten()[i]])
            distance.append(distances.flatten()[i])    

    m=pd.Series(movie,name='Movie_Id')
    d=pd.Series(distance,name='distance')
    recommend = pd.concat([m,d], axis=1)
    recommend = recommend.sort_values('distance',ascending=False)

    print('Recommendations for {0}:\n'.format(movie_user_mat.index[query_index]))
    

    for i in range(0,5):
        movie_list[name_movie].append({"Movie_Id": recommend["Movie_Id"].iloc[i], "distance": recommend["distance"].iloc[i]})
        print('{0}: {1}, with distance of {2}'.format(i, recommend["Movie_Id"].iloc[i], recommend["distance"].iloc[i]))

In [ ]:
## Backup
movie = []
distance = []

for i in range(0, len(distances.flatten())):
    if i != 0:
        movie.append(movie_user_mat.index[indices.flatten()[i]])
        distance.append(distances.flatten()[i])    

m=pd.Series(movie,name='Movie_Id')
d=pd.Series(distance,name='distance')
recommend = pd.concat([m,d], axis=1)
recommend = recommend.sort_values('distance',ascending=False)

print('Recommendations for {0}:\n'.format(movie_user_mat.index[query_index]))
for i in range(0,recommend.shape[0]):
    print('{0}: {1}, with distance of {2}'.format(i, recommend["Movie_Id"].iloc[i], recommend["distance"].iloc[i]))

###

In [ ]:
print(movie_list)

# Grundgedanke... Es gibt eine Matrix mit 17700 Filmen (Movie_IDs), jeder Film soll mit einem Title im DataFrame movie_titles verbunden werden. 
# Jeder Film soll einmal durch das Modell "K-NN" laufen und in einem neuem Dataframe gespeichert werden. Sodass ein neuer DataFrame entsteht, wo in der Erste Spalte der Film steht und in den weiteren 5 Spalten die Empfehlung + Distance

# zu jeden Film sollen 5 weitere Filme vorgeschlagen werden.! 

Kann vernachlässigt werden....


In [ ]:
# pivot and create movie-user matrix
#del data, 
movie_user_mat = df_filterd.pivot(index='Movie_Id', columns='Cust_Id', values='Rating').fillna(0)

# create mapper from movie title to index
movie_to_idx = {
   movie: i for i, movie in 
    enumerate(list(movie_titles.set_index('Movie_Id').loc[movie_user_mat.index].Name))
}
# transform matrix to scipy sparse matrix
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [ ]:
def fuzzy_matching(mapper, fav_movie, verbose=True):
    #https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
    #https://www.kaggle.com/code/razor08/knn-based-collaborative-filtering?scriptVersionId=81850169  Wo kommmt der Code her
    """
    return the closest match via fuzzy ratio. If no match found, return None
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True

    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]



def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    # fit
    model_knn.fit(data)
    # get input movie index
    print('You have input movie:', fav_movie)
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, reverse_mapper[idx], dist))

In [ ]:
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=120, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

In [ ]:
my_favorite = 'Haiku Tunnel'

make_recommendation(
    model_knn=model_knn,
    data=movie_user_mat_sparse,
    fav_movie=my_favorite,
    mapper=movie_to_idx,
    n_recommendations=5)

In [ ]:
## Läuft nicht!!!
# https://datascience.stackexchange.com/questions/52704/how-to-save-a-knn-model

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

"""
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=70, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)
"""
knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=70, n_jobs=-1)
knn.fit(movie_user_mat_sparse)

# Its important to use binary mode 
knnpickle_file = 'C:/Users/jsbreite/OneDrive - Jannis Breitenstein IT/Hochschule_Studium/5_Semester/Programmierprojekt/Netflix_Daten/KNN-Model.sav'
knnPickle = open('knnpickle_file', 'wb') 

# source, destination 
pickle.dump(knn, knnPickle)  

# close the file
knnPickle.close()
                
      
# load the model from disk
loaded_model = pickle.load(open('knnpickle_file', 'rb'))
result = loaded_model.predict(movie_user_mat_sparse) 